# Proyecto | Base de Datos de Big Data
---

**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4034.10 - Análisis de grandes volúmenes de datos**

Tecnológico de Monterrey

* Dr. Iván Olmos Pineda
* Mtra. Verónica Sandra Guzmán de Valle
* Mtro. Alberto Daniel Salinas Montemayor

**Proyecto**
Base de Datos de Big Data

---

**Equipo 37**

|  NOMBRE COMPLETO                        |     MATRÍCULA     |
| :-------------------------------------: |:-----------------:|
| Alejandro Díaz Villagómez               |  A01276769        |
| Alonso Pedrero Martínez                 |  A01769076        |
| César Iván Pedrero Martínez             |  A01366501        |
| Emiliano Saucedo Arriola                |  A01659258        |

# Amazon Reviews
---

### **Selección**

| **Campo**              | **Detalle**                                                                                  |
|-------------------------|----------------------------------------------------------------------------------------------|
| Nombre del dataset      | Amazon-reviews                                                                               |
| Enlace                  | [Amazon-reviews en Kaggle](https://www.kaggle.com/datasets/machharavikiran/amazon-reviews?resource=download) |
| Publicador              | Machha Ravi Kiran                                                                            |
| Última actualización    | Hace aproximadamente 2 años                                                                  |
| Formato - Tamaño        | CSV - 3.68 GB                                                                                 |
| Fuente                  | Repositorio Kaggle (acceso público)                                                           |


### **Atributos del Conjunto de Datos**

| Columna           | Tipo de dato | Descripción                                      |
|-------------------|--------------|--------------------------------------------------|
| marketplace       | string       | Mercado donde se realizó la compra               |
| customer_id       | integer      | Identificador único del cliente                        |
| review_id         | string       | Identificador único de la reseña                 |
| product_id        | string       | Identificador del producto                      |
| product_parent    | integer      | Identificador de grupo o "padre" para productos similares o relacionados             |
| product_title     | string       | Nombre del producto                              |
| product_category  | string       | Categoría del producto                           |
| star_rating       | integer      | Calificación otorgada por el cliente (1-5)             |
| helpful_votes     | integer      | Número de votos útiles recibidos por la reseña    |
| total_votes       | integer      | Número total de votos recibidos                  |
| vine              | string       | Participación en el programa Vine de reseñas (Y para sí, N para no)    |
| verified_purchase | string       | Indicador de compra verificada (Y para sí, N para no)                  |
| review_headline   | string       | Encabezado de la reseña                          |
| review_body       | string       | Cuerpo del texto de la reseña                    |
| review_date       | date         | Fecha en que se realizó la reseña                |
| sentiment         | integer      | Valor de sentimiento asignado a la reseña (1 para positivo, 0 para negativo)       |
  

### **Descripción General**
---

El conjunto de datos utilizado contiene reseñas de productos publicadas en Amazon, con un énfasis particular en artículos de la categoría electrónica. Cada registro incluye múltiples variables relevantes, tales como la calificación otorgada por el usuario (`star_rating`), la veracidad de la compra (`verified_purchase`), la participación en el programa Vine (vine), el contenido de la reseña (`review_body`, `review_headline`), y una etiqueta de sentimiento (`sentiment`) derivada.

Este dataset es adecuado para este proyecto por varias razones: su tamaño es considerable, lo cual permite aplicar técnicas de análisis de datos a gran escala; su estructura tabular es compatible con herramientas como PySpark; y proviene de una fuente reconocida (Kaggle), lo que respalda su validez y calidad. Estas características permiten realizar análisis de segmentación y muestreo para construir conjuntos de datos que representen adecuadamente distintos perfiles de usuarios y tipos de reseñas.

### **Descripción de Reglas de Particionamiento**
---

El proceso de particionamiento se diseñó a partir de tres variables categóricas clave del conjunto de datos:

* `star_rating`: calificación numérica del producto otorgada por el usuario (1 a 5),
* `verified_purchase`: indicador binario que refleja si la reseña proviene de una compra verificada (`"Y"` o `"N"`),
* `vine`: indicador de participación en el programa de reseñas Vine de Amazon (`"Y"` o `"N"`).

A partir de estas variables se generaron **20 combinaciones** distintas (5 × 2 × 2), cada una representando un subconjunto del conjunto de datos con características particulares. Por ejemplo:

* `R1_VPY_VY`: Calificación 1, compra verificada, usuario Vine.
* `R5_VPN_VN`: Calificación 5, compra no verificada, usuario no Vine.

Cada una de estas combinaciones fue utilizada como regla para filtrar el conjunto de datos y formar una partición independiente.

Estas reglas permiten observar el comportamiento del sentimiento de las reseñas bajo diferentes perfiles de usuario y producto, facilitando un análisis más granular. A cada partición se le asignó una clave en el formato:

```
R{star_rating}_VP{Y|N}_V{Y|N}
```

Las particiones con un volumen de datos suficiente fueron utilizadas para aplicar técnicas de muestreo estratificado basadas en la variable de salida `sentiment`.


## 0. Inicialización

> Tenga en cuenta que para ejecutar este notebook, deberá instalar algunas dependencias. Se recomienda ejecutar este comando para instalarlo en un entorno virtual. Para ello, siga este comando:

```bash
pip install findspark pyspark setuptools
```

In [1]:
import findspark
from pyspark.sql import SparkSession, functions as F
from itertools import product
from os import path

In [2]:
findspark.init()
findspark.find()

'/Users/alonsopedreromartinez/Documents/GitHub/TC4034.10-Equipo-37/act_1_big_data/lib/python3.12/site-packages/pyspark'

### Definición de clases auxiliares

In [3]:
class FileManager():
    @staticmethod
    def open_csv_file(input_path : str, file_name : str):
        """
        This method opens a csv file with pyspark
        """
        csv_df = spark.read.csv(
            path.join(input_path, file_name),
            header=True,
            inferSchema=True,
            multiLine=True,
            escape="\"",
            quote="\""
        )

        csv_df.show(truncate=20)

        return csv_df

In [4]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 14:00:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/04 14:00:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Cargar Dataset

In [5]:
PATH = "../files"
FILE = "amazon_electronics.csv"

In [6]:
df_reviews = FileManager.open_csv_file(PATH, FILE)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|sentiment|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|        1|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub 

## 1. Caracterización

### Análisis del mercado (`marketplace`)

Observaciones:
* El mercado se centra únicamente en US.

In [7]:
df_reviews.groupBy("marketplace").agg(
    F.count("*").alias("count")
).orderBy(F.desc("count")).show()


+-----------+-------+
|marketplace|  count|
+-----------+-------+
|         US|6906564|
+-----------+-------+



### Análisis de categorías (`product_category`)

Observaciones:
* Las reseñas pertecen solamente a la categoría de PC.

In [8]:
df_reviews.groupBy("product_category").agg(
    F.count("*").alias("count")
).orderBy(F.desc("count")).show()

+----------------+-------+
|product_category|  count|
+----------------+-------+
|              PC|6906564|
+----------------+-------+



### Análisis de clasificación de estrellas (`star_rating`)

Observaciones:
* Fuerte sesgo hacia reseñas positivas.
* Alrededor del 60% de las reseñas tienen 5 estrellas, lo que indica una tendencia general positiva.

In [9]:
star_rating_counts = df_reviews.groupBy("star_rating").count().orderBy("star_rating")

star_rating_stats = df_reviews.agg(
    F.min("star_rating"), F.max("star_rating"),
    F.avg("star_rating"), F.stddev("star_rating")
)

star_rating_counts.show()
star_rating_stats.show()

+-----------+-------+
|star_rating|  count|
+-----------+-------+
|          1| 756857|
|          2| 362156|
|          3| 513656|
|          4|1168208|
|          5|4105687|
+-----------+-------+



+----------------+----------------+-----------------+-------------------+
|min(star_rating)|max(star_rating)| avg(star_rating)|stddev(star_rating)|
+----------------+----------------+-----------------+-------------------+
|               1|               5|4.086460937739808|  1.362854001234071|
+----------------+----------------+-----------------+-------------------+



### Análisis de votos útiles (`helpful_ratio`)

Observaciones:
* Los datos sugieren una alta variabilidad en la utilidad percibida de las reseñas.

In [10]:
df_with_ratio = df_reviews.withColumn(
    "helpful_ratio",
    F.when(F.col("total_votes") > 0, F.col("helpful_votes") / F.col("total_votes")).otherwise(0)
)

helpful_ratio_stats = df_with_ratio.agg(
    F.min("helpful_ratio"), F.max("helpful_ratio"),
    F.avg("helpful_ratio"), F.stddev("helpful_ratio")
)

helpful_ratio_stats.show()

+------------------+------------------+-------------------+---------------------+
|min(helpful_ratio)|max(helpful_ratio)| avg(helpful_ratio)|stddev(helpful_ratio)|
+------------------+------------------+-------------------+---------------------+
|               0.0|               1.0|0.23128070845636955|   0.3967627166047604|
+------------------+------------------+-------------------+---------------------+



### Análisis de votos totales (`total_votes`)

Observaciones:
* En promedio, las reseñas reciben pocos votos (1.96).
* Lo anterior sugiere que son pocas las reseñas que reciben gran cantidad de votos.

In [11]:
total_votes_stats = df_with_ratio.agg(
    F.min("total_votes"), F.max("total_votes"),
    F.avg("total_votes"), F.stddev("total_votes")
)

total_votes_stats.show()

+----------------+----------------+------------------+-------------------+
|min(total_votes)|max(total_votes)|  avg(total_votes)|stddev(total_votes)|
+----------------+----------------+------------------+-------------------+
|               0|           48362|1.9620770907212328|  43.07079218821245|
+----------------+----------------+------------------+-------------------+



### Análisis de sentimiento (`sentiment`)

Observaciones:
* Se muestra una alta tendencia a sentimientos positivos. Posiblemente exista una alta correlación con el rating (`star_rating`).

In [ ]:
sentiment_counts = df_with_ratio.groupBy("sentiment").count().orderBy("sentiment")
sentiment_stats = df_with_ratio.agg(F.avg("sentiment").alias("positive_proportion"))

sentiment_counts.show()
sentiment_stats.show()

+---------+-------+
|sentiment|  count|
+---------+-------+
|        0|1632669|
|        1|5273895|
+---------+-------+



+-------------------+
|proporcion_positiva|
+-------------------+
| 0.7636061868101128|
+-------------------+



### Análisis de usuarios (Top 10 `customer_id`)

Observaciones:
* Algunos clientes han generado más de 350 reseñas, lo que podría señalar una actividad inusual o usuarios con cierta antigüedad en la plataforma.
* Esta exploración, permite comprender y detectar posibles sesgos individuales o comportamiento anómalo.

In [13]:
customer_top10 = df_with_ratio.groupBy("customer_id").count().orderBy(F.desc("count")).limit(10)
customer_top10.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   17957446|  458|
|   44834233|  442|
|   52938899|  366|
|   45664110|  275|
|   49452274|  261|
|   50820654|  256|
|   12200139|  251|
|   45070473|  251|
|   32038204|  241|
|   49266466|  240|
+-----------+-----+



### Análisis de vine

Observaciones:
* Son pocas las reseñas que fueron incentivadas por el programa Vine de Amazon.

In [14]:
vine_counts = df_with_ratio.groupBy("vine").count()
total_reviews = df_with_ratio.count()

vine_proportions = vine_counts.withColumn(
    "proportion", F.round(F.col("count") / F.lit(total_reviews), 4)
)
vine_proportions.show()

+----+-------+----------+
|vine|  count|proportion|
+----+-------+----------+
|   Y|  36228|    0.0052|
|   N|6870336|    0.9948|
+----+-------+----------+



### Análisis de compras verificadas (`verified_purchase`)

Observaciones:
* Se puede apreciar una alta proporción de compras verificadas, lo cual sugiere la credibilidad de las reseñas provenientes de usuarios que realmente compraron el producto.

In [15]:
verified_counts = df_with_ratio.groupBy("verified_purchase").count()
verified_proportions = verified_counts.withColumn(
    "proportion", F.round(F.col("count") / F.lit(total_reviews), 4)
)

verified_proportions.show()

+-----------------+-------+----------+
|verified_purchase|  count|proportion|
+-----------------+-------+----------+
|                Y|6047075|    0.8756|
|                N| 859489|    0.1244|
+-----------------+-------+----------+



### Análisis temporal por año (`review_year`)

Observaciones:
* Las reseñas han incrementado a través del tiempo, con un fuerte crecimiento a partir de 2013.

In [16]:
df_with_year = df_with_ratio.withColumn("review_year", F.year("review_date"))
reviews_by_year = df_with_year.groupBy("review_year").count().orderBy("review_year")

reviews_by_year.show()

+-----------+-------+
|review_year|  count|
+-----------+-------+
|       1999|    384|
|       2000|   3596|
|       2001|   6588|
|       2002|  10125|
|       2003|  13619|
|       2004|  14124|
|       2005|  18171|
|       2006|  26277|
|       2007|  59870|
|       2008|  81409|
|       2009| 129840|
|       2010| 213170|
|       2011| 397182|
|       2012| 661742|
|       2013|1396819|
|       2014|1996666|
|       2015|1876982|
+-----------+-------+



Las variables de interés se seleccionaron con base en su relación con la percepción del cliente y la confiabilidad de la reseña. En particular, `star_rating` y `verified_purchase` mostraron una buena distribución de combinaciones y relevancia para el objetivo del proyecto, que es analizar el sentimiento de los clientes.

## 2. Particionamiento

A partir de las variables de caracterización seleccionadas (`star_rating`, `verified_purchase` y `vine`), se construyeron combinaciones de particionamiento. Cada combinación representa un subconjunto de la base de datos que cumple simultáneamente con valores específicos de estas tres variables.

Se generaron 20 combinaciones posibles:

- 5 valores para star_rating: {1, 2, 3, 4, 5}
- 2 valores para verified_purchase: {Y, N}
- 2 valores para vine: {Y, N}

Total de combinaciones: 5 × 2 × 2 = 20

Estas combinaciones permiten observar el comportamiento del sentimiento bajo distintos contextos de calificación y tipo de reseña. Por ejemplo, es posible estudiar si las reseñas de usuarios con compras verificadas tienden a tener un tono más positivo que aquellas sin verificación.

In [17]:
# Previously defined relevant columns for the activity.
RELEVANT_COLUMNS_FOR_CHARACTERIZATION = [
  "star_rating",
  "helpful_votes",
  "total_votes",
  "vine",
  "verified_purchase",
  "review_date",
  "sentiment"
]

In [18]:
df_reviews_filtered = df_reviews.select(*RELEVANT_COLUMNS_FOR_CHARACTERIZATION)

### Generación de las particiones

Se implementó un procedimiento automático en PySpark que:

1. Filtra los registros de la base de datos que cumplen con cada combinación de valores.
2. Almacena cada subconjunto en un diccionario indexado por nombre de combinación (ej. "R5_VPY_VN" para `star_rating`=5, `verified_purchase`=Y, `vine`=N).
3. Imprime la cantidad de registros por partición para control y trazabilidad.

Las particiones con muy pocos registros pueden ser descartadas en etapas posteriores para evitar problemas en el análisis.

In [19]:
class PartitioningManager:

    @staticmethod
    def compute_probabilities(df, cols):
        """
        Computes and returns the probability of each combination of values in the specified columns.
        """
        total_count = df.count()
        return df.groupBy(cols).count() \
                 .withColumn("probability", F.round(F.col("count") / total_count, 6)) \
                 .orderBy("probability", ascending=False)

    @staticmethod
    def filter_partition(df, star_rating, verified_purchase, vine):
        """
        Filters the DataFrame by specific values for rating, verified purchase, and vine.
        """
        return df.filter(
            (F.col("star_rating") == star_rating) &
            (F.col("verified_purchase") == verified_purchase) &
            (F.col("vine") == vine)
        )

    @staticmethod
    def generate_all_partitions(df, min_probability=0.0001):
        """
        Generates partitions only for combinations whose joint probability is above min_probability.
        """

        prob_df = PartitioningManager.compute_probabilities(
            df, ["star_rating", "verified_purchase", "vine"]
        )

        filtered_combinations = prob_df.filter(
            F.col("probability") >= min_probability
        ).select("star_rating", "verified_purchase", "vine").collect()

        partitions = {}
        for row in filtered_combinations:
            rating = row["star_rating"]
            purchase = row["verified_purchase"]
            vine = row["vine"]

            key = f"R{rating}_VP{purchase}_V{vine}"
            filtered = PartitioningManager.filter_partition(df, rating, purchase, vine)
            partitions[key] = filtered
            print(f"Partition {key} created with {filtered.count()} records.")

        return partitions

    @staticmethod
    def stratified_sample_partitioned_data(partitions_dict, label_col="sentiment", fraction=0.3, min_rows=50):
        """
        Applies stratified sampling to each partition based on sentiment.
        """
        sampled_partitions = {}

        for key, df in partitions_dict.items():
            count = df.count()

            if count < min_rows:
                print(f"Skipping partition {key} — only {count} rows (<{min_rows})")
                continue

            sentiments = df.select(label_col).distinct().rdd.flatMap(lambda x: x).collect()
            fractions = {s: fraction for s in sentiments}

            sampled_df = df.sampleBy(label_col, fractions, seed=42)
            sampled_partitions[key] = sampled_df
            print(f"Sampled {sampled_df.count()} rows from partition {key} (original: {count})")

        return sampled_partitions


In [20]:
partitions = PartitioningManager.generate_all_partitions(df_reviews, min_probability=0.00001)

Partition R5_VPY_VN created with 3679909 records.


Partition R4_VPY_VN created with 1019728 records.


Partition R1_VPY_VN created with 603371 records.


Partition R3_VPY_VN created with 443364 records.


Partition R5_VPN_VN created with 410073 records.


...

Partition R4_VPN_VY created with 13240 records.


Partition R3_VPN_VY created with 4886 records.


Partition R2_VPN_VY created with 1634 records.


Partition R1_VPN_VY created with 705 records.


Partition R5_VPY_VY created with 101 records.


In [21]:
# Sample of one of the generated partitions.
partitions["R5_VPY_VN"].show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|sentiment|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|        1|
|         US|   38264512|R39NJY2YJ1JFSV|B00AQMTND2|     964759214|Aleratec SATA 

## 3.Técnica de muestreo aplicada por partición

Una vez construidas las particiones, se aplicó una técnica de **muestreo estratificado** sobre cada subconjunto, usando la variable `sentiment` como variable de estratificación. Esto asegura que cada muestra mantenga la proporción original de clases de sentimiento en la partición.

Para evitar particiones con tamaños insuficientes, se definió un umbral mínimo (`min_rows`) que descarta automáticamente las particiones con muy pocos registros.

Además, se permitió configurar el porcentaje de muestreo (`fraction`) por clase de sentimiento. Esto ofrece flexibilidad para ajustar el tamaño del conjunto de entrenamiento o validación según necesidades posteriores.

#### Justificación del muestreo estratificado

* **Preservación del equilibrio de clases**: Al muestrear por clase de sentimiento se evitan sesgos por clases desbalanceadas.
* **Relevancia contextual**: Al aplicar el muestreo dentro de cada partición (y no sobre la base completa), se conserva la variabilidad contextual de las reseñas.
* **Evita el submuestreo accidental**: Las particiones muy pequeñas son descartadas de forma controlada, garantizando que el conjunto final tenga representatividad suficiente.

In [22]:
sampled_partitions = PartitioningManager.stratified_sample_partitioned_data(partitions, fraction=0.4, min_rows=100)

Sampled 1473392 rows from partition R5_VPY_VN (original: 3679909)


Sampled 408759 rows from partition R4_VPY_VN (original: 1019728)


Sampled 241961 rows from partition R1_VPY_VN (original: 603371)


Sampled 177507 rows from partition R3_VPY_VN (original: 443364)


Sampled 164237 rows from partition R5_VPN_VN (original: 410073)


...

Sampled 5385 rows from partition R4_VPN_VY (original: 13240)


Sampled 1979 rows from partition R3_VPN_VY (original: 4886)


Sampled 682 rows from partition R2_VPN_VY (original: 1634)


Sampled 294 rows from partition R1_VPN_VY (original: 705)


Sampled 35 rows from partition R5_VPY_VY (original: 101)


Para la recuperación de instancias desde cada partición generada, se empleó un **muestreo estratificado con fracción del 20%**, aplicado sobre la variable de salida `sentiment`. Esta técnica permite seleccionar una submuestra proporcional de registros pertenecientes a cada clase de sentimiento, conservando la distribución original presente en cada partición.

Con ello, se garantiza que el conjunto muestreado sea estadísticamente representativo, lo cual es importante para el desarrollo posterior de modelos de aprendizaje supervisado. Este enfoque también permite mitigar sesgos introducidos por clases dominantes y asegurar la diversidad de ejemplos en cada subconjunto. Además, se filtraron las particiones con un volumen menor a 50 observaciones para evitar problemas derivados de tamaños de muestra insuficientes.
